In [ ]:
#!pip install ipwhois
#!pip install IPy

In [54]:
import pandas as pd
import os
import ipwhois
from tqdm import tqdm
import IPy
from sklearn.model_selection import train_test_split
import xgboost
from sklearn import metrics
from sklearn.utils import class_weight
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import time
import datetime
from sklearn.externals import joblib

PATH = r'D:\Documents\ITC\Workshops\Checkpoint'
file_benign = 'Train_Benign_Traffic.csv'
file_detection = 'Train_Detections.csv'
file_train_ips = 'Train_IPS.csv'

In [34]:
train_benign = pd.read_csv(os.path.join(PATH, file_benign), encoding = 'latin1')
train_benign.head()

,Unnamed: 0,app_name,app_risk,client,dst_ip,dst_port,epoch_time,protocol,received_bytes,sent_bytes,src_ip
0,0,Google Play,Low,2,222.58.210.46,443.0,1547242292,TCP,44096,67712,10.197.88.33
1,1,Twitter,Low,5,110.244.46.16,443.0,1547468725,TCP,7712,1990,10.0.11.69
2,2,Apple Siri,Low,2,23.130.21.5,443.0,1547415821,TCP,12928,8464,192.168.174.137
3,3,Microsoft Account,Very Low,2,71.55.163.78,443.0,1547152973,TCP,19808,4376,192.168.190.183
4,4,Unknown Traffic,Unknown,5,230.0.0.1,NaN,1547070189,TCP,0,252,10.0.11.2


In [35]:
train_detection = pd.read_csv(os.path.join(PATH, file_detection))
train_detection.head()
train_detection.client.unique()

array([3, 2, 1, 5], dtype=int64)

In [36]:
train_ips = pd.read_csv(os.path.join(PATH, file_train_ips))
train_ips.head()

,Unnamed: 0,Unnamed: 0.1,attack,client,dst_ip,severity,src_ip
0,0,1039,NaN,2,10.200.52.23,Informational,29.44.170.65
1,1,2509,SSL Enforcement Violation,1,23.248.146.149,High,200.75.192.252
2,2,256,NaN,2,10.197.206.52,Informational,58.7.22.121
3,3,1942,NaN,5,157.101.250.49,Informational,10.0.11.142
4,4,1184,DNS Enforcement Violation,2,10.200.105.201,Critical,10.195.142.73


# Client 1

In [37]:
train_b1 = train_benign[train_benign['client'].values == 1]
train_d1 = train_detection[train_detection['client'].values == 1]
train_ips1 = train_ips[train_ips['client'].values == 1]

In [38]:
train_b1['label'] = 'benign'
train_d1['label'] = 'malignant'


C:\Users\lastm\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\lastm\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [39]:
train_b1 = train_b1.drop(['app_name', 'app_risk', 'epoch_time', 'Unnamed: 0'], axis = 1)
train_d1 = train_d1.drop(['epoch_time', 'Unnamed: 0'], axis=1)

In [40]:
train_b1.columns

Index(['client', 'dst_ip', 'dst_port', 'protocol', 'received_bytes',
       'sent_bytes', 'src_ip', 'label'],
      dtype='object')

In [41]:
train_d1.columns

Index(['client', 'dst_ip', 'dst_port', 'label', 'protocol', 'received_bytes',
       'sent_bytes', 'src_ip'],
      dtype='object')

# Try with balanced df

In [42]:
train_b = list()
train_d = list()
train_i = list()

for i in range(1,6):
    
    train_b.append(train_benign[train_benign['client'].values == i])
    train_d.append(train_detection[train_detection['client'].values == i])
    train_i.append(train_ips[train_ips['client'].values == i])

# Client 2

In [43]:
i=1
len(train_d[i])
train_d[i].head()

,Unnamed: 0,client,dst_ip,dst_port,epoch_time,label,protocol,received_bytes,sent_bytes,src_ip
1,1,2,8.8.8.8,53,1546607964,Malware,UDP,0,0,192.168.186.37
2,2,2,10.200.105.201,53,1545917481,Malware,UDP,456,164,10.195.74.11
3,3,2,8.8.8.8,53,1545386094,Malware,UDP,0,70,192.168.143.103
5,5,2,212.189.218.209,80,1545512693,Malware,TCP,415,1000,10.196.74.13
7,7,2,10.200.105.202,53,1545902168,Malware,UDP,0,0,10.200.43.110


In [44]:
balanced_benign = train_b[i].sample(n=len(train_d[i]))
balanced_benign.head()

,Unnamed: 0,app_name,app_risk,client,dst_ip,dst_port,epoch_time,protocol,received_bytes,sent_bytes,src_ip
40402,40402,Gmail,Medium,2,222.58.198.165,443.0,1547153063,TCP,1130496,292352,10.197.208.98
37790,37790,MSN-web,Low,2,179.222.34.50,80.0,1547241348,TCP,19552,4696,10.200.52.96
39174,39174,Office365-Outlook-web,Very Low,2,58.97.146.98,443.0,1547408823,TCP,3121152,840704,10.197.208.21
2878,2878,Meraki Cloud Controller,Low,2,215.206.58.5,80.0,1547396697,TCP,19776,5112,10.197.254.200
21645,21645,Unknown Traffic,Unknown,2,10.197.81.254,192.0,1547322760,UDP,0,0,10.197.80.83


In [45]:
balanced_benign = balanced_benign.drop(['app_name', 'app_risk', 'epoch_time', 'Unnamed: 0'], axis = 1)
balanced_benign['label'] = 'benign'
train_d[i] = train_d[i].drop(['epoch_time', 'Unnamed: 0'], axis=1)
train_d[i]['label'] = 'malignant'

df_balanced = pd.concat([balanced_benign, train_d[i]], ignore_index=True)
df_balanced = pd.concat([df_balanced, pd.get_dummies(df_balanced.protocol)], axis=1, ignore_index=True)
df_balanced.head()

C:\Users\lastm\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,0,1,2,3,4,5,6,7,8,9
0,2,222.58.198.165,443.0,benign,TCP,1130496,292352,10.197.208.98,1,0
1,2,179.222.34.50,80.0,benign,TCP,19552,4696,10.200.52.96,1,0
2,2,58.97.146.98,443.0,benign,TCP,3121152,840704,10.197.208.21,1,0
3,2,215.206.58.5,80.0,benign,TCP,19776,5112,10.197.254.200,1,0
4,2,10.197.81.254,192.0,benign,UDP,0,0,10.197.80.83,0,1


In [46]:
df_balanced.columns = ['client', 'dst_ip', 'dst_port', 'label', 'protocol', 'received_bytes', 'sent_bytes', 'src_ip', 'TCP', 'UDP']
df_balanced = df_balanced.drop(['protocol'], axis = 1)
df_balanced.head()

,client,dst_ip,dst_port,label,received_bytes,sent_bytes,src_ip,TCP,UDP
0,2,222.58.198.165,443.0,benign,1130496,292352,10.197.208.98,1,0
1,2,179.222.34.50,80.0,benign,19552,4696,10.200.52.96,1,0
2,2,58.97.146.98,443.0,benign,3121152,840704,10.197.208.21,1,0
3,2,215.206.58.5,80.0,benign,19776,5112,10.197.254.200,1,0
4,2,10.197.81.254,192.0,benign,0,0,10.197.80.83,0,1


In [47]:
def create_ip_features(ip):

    res = ip.split('.')
    for ind,val in enumerate(res):
        if len(val)<3:
            res[ind]='0'*(3-len(val)) + val
        else:
            continue

    return res

In [48]:
dst_ip_1 = list()
dst_ip_2 = list()
dst_ip_3 = list()
dst_ip_4 = list()

src_ip_1 = list()
src_ip_2 = list()
src_ip_3 = list()
src_ip_4 = list()

for ip in df_balanced.dst_ip:
    res = create_ip_features(ip)
    dst_ip_1.append(res[0])
    dst_ip_2.append(res[1])
    dst_ip_3.append(res[2])
    dst_ip_4.append(res[3])

for ip in df_balanced.src_ip:
    res = create_ip_features(ip)
    src_ip_1.append(res[0])
    src_ip_2.append(res[1])
    src_ip_3.append(res[2])
    src_ip_4.append(res[3])
    
    

In [49]:
# df_balanced = df_balanced.drop(['dst_ip', 'src_ip'], axis=1)
df_balanced['dst_ip_1'] = dst_ip_1
df_balanced['dst_ip_2'] = dst_ip_2
df_balanced['dst_ip_3'] = dst_ip_3
df_balanced['dst_ip_4'] = dst_ip_4
df_balanced['src_ip_1'] = src_ip_1
df_balanced['src_ip_2'] = src_ip_2
df_balanced['src_ip_3'] = src_ip_3
df_balanced['src_ip_4'] = src_ip_4
df_balanced.head()

,client,dst_ip,dst_port,label,received_bytes,sent_bytes,src_ip,TCP,UDP,dst_ip_1,dst_ip_2,dst_ip_3,dst_ip_4,src_ip_1,src_ip_2,src_ip_3,src_ip_4
0,2,222.58.198.165,443.0,benign,1130496,292352,10.197.208.98,1,0,222,058,198,165,010,197,208,098
1,2,179.222.34.50,80.0,benign,19552,4696,10.200.52.96,1,0,179,222,034,050,010,200,052,096
2,2,58.97.146.98,443.0,benign,3121152,840704,10.197.208.21,1,0,058,097,146,098,010,197,208,021
3,2,215.206.58.5,80.0,benign,19776,5112,10.197.254.200,1,0,215,206,058,005,010,197,254,200
4,2,10.197.81.254,192.0,benign,0,0,10.197.80.83,0,1,010,197,081,254,010,197,080,083


In [50]:
# log column
df_balanced['diff'] = np.abs(df_balanced['received_bytes'] - df_balanced['sent_bytes'])
df_balanced['log received_bytes'] = df_balanced['received_bytes'].apply(lambda x: np.log(x))
df_balanced['log sent_bytes'] = df_balanced['sent_bytes'].apply(lambda x: np.log(x))

df_balanced = df_balanced.replace([np.inf, -np.inf], np.nan)
df_balanced = df_balanced.dropna()

In [51]:
X = df_balanced.drop(['label'], axis =1)
X = pd.get_dummies(X)
y = df_balanced['label'].values #.map(lambda x: 1 if x == 'malignat' else 0).values
# y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [52]:
# model = xgboost.XGBClassifier() #scale_pos_weight=sum(y_train == 'benign')/sum(y_train == 'malignant'))
model = RandomForestClassifier(class_weight='balanced')

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('f1 score is {}'.format(metrics.f1_score(y_test, y_pred, pos_label='malignant')))
print('accuracy is {}%'.format(metrics.accuracy_score(y_test,y_pred)*100))
print('precision is {}'.format(metrics.precision_score(y_test, y_pred, pos_label='malignant')))

C:\Users\lastm\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


f1 score is 0.9411764705882353
accuracy is 96.55172413793103%
precision is 1.0


In [55]:
filename = 'finalized_model_precision1.sav'
joblib.dump(model, filename)

['finalized_model_precision1.sav']

In [56]:
# adding internal external 
int_ext = pd.read_csv('D:\Documents\ITC\Workshops\Checkpoint\hackathon\cphackathon\Data\ErichData_with_time.csv')
int_ext = int_ext[int_ext.client == 2]
int_ext.head()

,Unnamed: 0,Unnamed: 0.1,client,dst_ip,dst_port,epoch_time,label,protocol,received_bytes,sent_bytes,...,type_dst,subtype_dst,type_src,subtype_src,log_sent_bytes,log_received_bytes,real time,year,month,day
0,0,0,2,222.58.210.46,443,1547242292,benign,1,44096,67712,...,Internal,Class A,Internal,Class A,11,10,2019-01-11 21:31:32,2019,1,11
2,2,2,2,23.130.21.5,443,1547415821,benign,1,12928,8464,...,Internal,Class C,Internal,Class C,9,9,2019-01-13 21:43:41,2019,1,13
3,3,3,2,71.55.163.78,443,1547152973,benign,1,19808,4376,...,Internal,Class C,Internal,Class C,8,9,2019-01-10 20:42:53,2019,1,10
5,5,5,2,19.107.4.52,80,1547065825,benign,1,1013,283,...,Internal,Class A,Internal,Class A,5,6,2019-01-09 20:30:25,2019,1,9
6,6,6,2,10.177.66.94,-1,1547153345,benign,1,7896,7960,...,Internal,Class A,Internal,Class A,8,8,2019-01-10 20:49:05,2019,1,10


In [57]:
def ip_internal_external(ip_addresses):
    results = {'ip':[], 'type':[], 'subtype':[]}
    for ip_address in ip_addresses:
        results['ip'].append(ip_address)
        splitted_ip = ip_address.split('.')
        if splitted_ip[0] == '127':
            results['type'].append('Loopback')
            results['subtype'].append('Loopback')
        elif int(splitted_ip[0]) >= 224 and int(splitted_ip[0]) <= 239:
            results['type'].append('Multicast')
            results['subtype'].append('Multicast')
        elif ip_address == '255.255.255.255':
            results['type'].append('Limited Broadcast')
            results['subtype'].append('Limited Broadcast')
        elif splitted_ip[0] == '10':
            results['type'].append('Internal')
            results['subtype'].append('Class A')
        elif splitted_ip[0] == '172' and int(splitted_ip[1]) >= 16 and int(splitted_ip[1]) <= 31:
            results['type'].append('Internal')
            results['subtype'].append('Class B')
        elif splitted_ip[0] == '192' and splitted_ip[1] == '168':
            results['type'].append('Internal')
            results['subtype'].append('Class C')
        elif splitted_ip[3] == '0':
            results['type'].append('Network space')
            results['subtype'].append('Network space')
        elif splitted_ip[3] == '255':
            results['type'].append('Broadcast address')
            results['subtype'].append('Broadcast address')
        else:
            results['type'].append('External')
            results['subtype'].append('External')
    
    return pd.DataFrame.from_dict(results)
            
# ip_internal_external(['192.168.1.1', '20.10.30.40', '10.1.2.3', '172.25.30.8', '254.255.255.255',
#                      '255.255.255.255', '230.28.30.21', '120.0.0.0', '127.5.6.7'])


In [64]:
enrich_src = ip_internal_external(df_balanced.src_ip)
enrich_dst = ip_internal_external(df_balanced.dst_ip) 

In [66]:
enrich_src.head()

,ip,type,subtype
0,10.197.208.98,Internal,Class A
1,10.200.52.96,Internal,Class A
2,10.197.208.21,Internal,Class A
3,10.197.254.200,Internal,Class A
4,10.196.94.58,Internal,Class A


In [67]:
df_balanced['src_type'] = enrich_src.type
df_balanced['src_subtype'] = enrich_src.subtype
df_balanced['dst_type'] = enrich_dst.type
df_balanced['dst_subtype'] = enrich_dst.subtype

In [68]:
X = df_balanced.drop(['label'], axis =1)
X = pd.get_dummies(X)
y = df_balanced['label'].values #.map(lambda x: 1 if x == 'malignat' else 0).values
# y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [69]:
model = RandomForestClassifier(class_weight='balanced')

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('f1 score is {}'.format(metrics.f1_score(y_test, y_pred, pos_label='malignant')))
print('accuracy is {}%'.format(metrics.accuracy_score(y_test,y_pred)*100))
print('precision is {}'.format(metrics.precision_score(y_test, y_pred, pos_label='malignant')))

C:\Users\lastm\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


f1 score is 0.9545454545454545
accuracy is 96.55172413793103%
precision is 1.0


In [70]:
filename = 'finalized_model_precision1_internalexternal.sav'
joblib.dump(model, filename)

['finalized_model_precision1_internalexternal.sav']

# Unbalanced client 1

In [71]:
df = pd.concat([train_b1 , train_d1], ignore_index=True)
df.head()

C:\Users\lastm\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,client,dst_ip,dst_port,label,protocol,received_bytes,sent_bytes,src_ip
0,1,218.82.101.50,993.0,benign,TCP,344576,71296,200.75.192.252
1,1,14.39.55.190,443.0,benign,TCP,8992,8832,200.75.192.252
2,1,52.30.14.249,80.0,benign,TCP,246,325,200.75.192.252
3,1,149.204.212.210,443.0,benign,TCP,38797312,5062656,200.75.192.252
4,1,200.75.192.252,80.0,benign,TCP,522,824,199.112.224.171


In [72]:
df = pd.concat([df, pd.get_dummies(df.protocol)], axis=1, ignore_index=True)
df.columns = ['client', 'dst_ip', 'dst_port', 'label', 'protocol', 'received_bytes', 'sent_bytes', 'src_ip', 'TCP', 'UDP']
df = df.drop(['protocol'], axis = 1)
df.head()

,client,dst_ip,dst_port,label,received_bytes,sent_bytes,src_ip,TCP,UDP
0,1,218.82.101.50,993.0,benign,344576,71296,200.75.192.252,1,0
1,1,14.39.55.190,443.0,benign,8992,8832,200.75.192.252,1,0
2,1,52.30.14.249,80.0,benign,246,325,200.75.192.252,1,0
3,1,149.204.212.210,443.0,benign,38797312,5062656,200.75.192.252,1,0
4,1,200.75.192.252,80.0,benign,522,824,199.112.224.171,1,0


In [73]:

def create_ip_features(ip):

    res = ip.split('.')
    for ind,val in enumerate(res):
        if len(val)<3:
            res[ind]='0'*(3-len(val)) + val
        else:
            continue

    return res

In [36]:
# for index in df.index:

#     ip = create_ip_features(df.loc[index, 'src_ip'])
#     df.loc[i,'src_1'] = ip[0]
#     df.loc[i,'src_2'] = ip[1] 
#     df.loc[i,'src_3'] = ip[2] 
#     df.loc[i,'src_4'] = ip[3] 

#     ip = create_ip_features(df.loc[index, 'dst_ip'])
#     df.loc[i,'dst_1'] = ip[0]
#     df.loc[i,'dst_2'] = ip[1] 
#     df.loc[i,'dst_3'] = ip[2] 
#     df.loc[i,'dst_4'] = ip[3]    

# df = df.drop(['dst_ip', 'src_ip'], axis = 1)
# df.head()
    

In [74]:
dst_ip_1 = list()
dst_ip_2 = list()
dst_ip_3 = list()
dst_ip_4 = list()

src_ip_1 = list()
src_ip_2 = list()
src_ip_3 = list()
src_ip_4 = list()

for ip in df.dst_ip:
    res = create_ip_features(ip)
    dst_ip_1.append(res[0])
    dst_ip_2.append(res[1])
    dst_ip_3.append(res[2])
    dst_ip_4.append(res[3])

for ip in df.src_ip:
    res = create_ip_features(ip)
    src_ip_1.append(res[0])
    src_ip_2.append(res[1])
    src_ip_3.append(res[2])
    src_ip_4.append(res[3])
    
    

In [75]:
# df = df.drop(['dst_ip', 'src_ip'], axis=1)
df['dst_ip_1'] = dst_ip_1
df['dst_ip_2'] = dst_ip_2
df['dst_ip_3'] = dst_ip_3
df['dst_ip_4'] = dst_ip_4
df['src_ip_1'] = src_ip_1
df['src_ip_2'] = src_ip_2
df['src_ip_3'] = src_ip_3
df['src_ip_4'] = src_ip_4
df.head()

,client,dst_ip,dst_port,label,received_bytes,sent_bytes,src_ip,TCP,UDP,dst_ip_1,dst_ip_2,dst_ip_3,dst_ip_4,src_ip_1,src_ip_2,src_ip_3,src_ip_4
0,1,218.82.101.50,993.0,benign,344576,71296,200.75.192.252,1,0,218,082,101,050,200,075,192,252
1,1,14.39.55.190,443.0,benign,8992,8832,200.75.192.252,1,0,014,039,055,190,200,075,192,252
2,1,52.30.14.249,80.0,benign,246,325,200.75.192.252,1,0,052,030,014,249,200,075,192,252
3,1,149.204.212.210,443.0,benign,38797312,5062656,200.75.192.252,1,0,149,204,212,210,200,075,192,252
4,1,200.75.192.252,80.0,benign,522,824,199.112.224.171,1,0,200,075,192,252,199,112,224,171


In [76]:
enrich_src = ip_internal_external(df.src_ip)
enrich_dst = ip_internal_external(df.dst_ip) 

In [77]:
df['src_type'] = enrich_src.type
df['src_subtype'] = enrich_src.subtype
df['dst_type'] = enrich_dst.type
df['dst_subtype'] = enrich_dst.subtype

In [80]:
df = df.dropna(axis=0)

In [81]:
X = df.drop(['label'], axis =1)
X = pd.get_dummies(X)
y = df['label'].values #.map(lambda x: 1 if x == 'malignat' else 0).values
# y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [82]:
# model = xgboost.XGBClassifier() #scale_pos_weight=sum(y_train == 'benign')/sum(y_train == 'malignant'))
# model = RandomForestClassifier(class_weight='balanced')

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('f1 score is {}'.format(metrics.f1_score(y_test, y_pred, pos_label='malignant')))
print('accuracy is {}%'.format(metrics.accuracy_score(y_test,y_pred)*100))
print('precision is {}'.format(metrics.precision_score(y_test, y_pred, pos_label='malignant')))

C:\Users\lastm\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


f1 score is 0.0
accuracy is 99.88193624557262%


C:\Users\lastm\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


precision is 0.0


In [83]:
print(metrics.classification_report(y_test, y_pred))

C:\Users\lastm\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

      benign       1.00      1.00      1.00       846
   malignant       0.00      0.00      0.00         1

   micro avg       1.00      1.00      1.00       847
   macro avg       0.50      0.50      0.50       847
weighted avg       1.00      1.00      1.00       847



# Adding column

In [44]:
# log column
df['diff'] = np.abs(df['received_bytes'] - df['sent_bytes'])
df['received_bytes'] = df['received_bytes'].apply(lambda x: np.log(x))
df['sent_bytes'] = df['sent_bytes'].apply(lambda x: np.log(x))

df = df.dropna()

In [45]:
X = df.drop(['label'], axis =1)
X = pd.get_dummies(X)
y = df['label'].values #.map(lambda x: 1 if x == 'malignat' else 0).values
# y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [46]:
model = RandomForestClassifier(class_weight='balanced')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print('f1 score is {}'.format(metrics.f1_score(y_test, y_pred, pos_label='malignant')))
print('accuracy is {}%'.format(metrics.accuracy_score(y_test,y_pred)*100))

C:\Users\lastm\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


f1 score is 0.0
accuracy is 100.0%


In [32]:
df.head()

,client,dst_port,label,received_bytes,sent_bytes,TCP,UDP,dst_ip_1,dst_ip_2,dst_ip_3,dst_ip_4,src_ip_1,src_ip_2,src_ip_3,src_ip_4,diff
0,1,993.0,benign,2.545537,2.413643,1,0,218,082,101,050,200,075,192,252,0.131894
1,1,443.0,benign,2.208724,2.206750,1,0,014,039,055,190,200,075,192,252,0.001974
2,1,80.0,benign,1.705717,1.755065,1,0,052,030,014,249,200,075,192,252,0.049348
3,1,443.0,benign,2.860706,2.736793,1,0,149,204,212,210,200,075,192,252,0.123913
4,1,80.0,benign,1.833808,1.904220,1,0,200,075,192,252,199,112,224,171,0.070413


# adding the risk column

In [ ]:
d_score = dict()

for ip in tqdm(train_benign.dst_ip.values):
    d_score[ip] = len(train_detection[train_detection.dst_ip == ip]) / len(train_benign[train_benign == ip])

In [ ]:
for i in tqdm(df.index):
    df.loc[i,'risk_score'] = d_score[df.loc[i,'dst_ip']]
    

# Comparison epoch

In [58]:
train_epoch_benign = train_benign.drop(['app_name', 'app_risk', 'Unnamed: 0'], axis = 1)
train_epoch_detection = train_detection.drop(['Unnamed: 0'], axis=1)
train_epoch_benign['label'] = 'benign'

df_epoch = pd.concat([train_epoch_benign, train_epoch_detection], ignore_index=True)
df_epoch.head()

C:\Users\lastm\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


,client,dst_ip,dst_port,epoch_time,label,protocol,received_bytes,sent_bytes,src_ip
0,2,222.58.210.46,443.0,1547242292,benign,TCP,44096,67712,10.197.88.33
1,5,110.244.46.16,443.0,1547468725,benign,TCP,7712,1990,10.0.11.69
2,2,23.130.21.5,443.0,1547415821,benign,TCP,12928,8464,192.168.174.137
3,2,71.55.163.78,443.0,1547152973,benign,TCP,19808,4376,192.168.190.183
4,5,230.0.0.1,NaN,1547070189,benign,TCP,0,252,10.0.11.2


In [72]:
df_epoch['real time']=df_epoch['epoch_time'].apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)))

In [77]:
df_epoch['month'] = df_epoch['real time'].apply(lambda x: x.month)
df_epoch.head()

,client,dst_ip,dst_port,epoch_time,label,protocol,received_bytes,sent_bytes,src_ip,real time,month
0,2,222.58.210.46,443.0,1547242292,benign,TCP,44096,67712,10.197.88.33,2019-01-11 21:31:32,1
1,5,110.244.46.16,443.0,1547468725,benign,TCP,7712,1990,10.0.11.69,2019-01-14 12:25:25,1
2,2,23.130.21.5,443.0,1547415821,benign,TCP,12928,8464,192.168.174.137,2019-01-13 21:43:41,1
3,2,71.55.163.78,443.0,1547152973,benign,TCP,19808,4376,192.168.190.183,2019-01-10 20:42:53,1
4,5,230.0.0.1,NaN,1547070189,benign,TCP,0,252,10.0.11.2,2019-01-09 21:43:09,1


In [ ]:

error_count = 0 
result_list =list()

for ip in tqdm(train_detection.src_ip):
    test = IPy.IP(ip)
    if test.iptype() == 'PRIVATE':
        continue
    else:
        
        try:
            obj = ipwhois.IPWhois(ip)
            results = obj.lookup_rdap(depth=1)

        except ipwhois.exceptions.IPDefinedError:
            error_count += 1
        else:
            result_list.append(results)

In [ ]:
error_count

In [ ]:
len(result_list)

In [ ]:
result_list[5]
    